In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Found existing installation: unsloth 2024.11.7
Uninstalling unsloth-2024.11.7:
  Successfully uninstalled unsloth-2024.11.7
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-takgaimd/unsloth_82cb18b94a304bc9ba71989233d4ea2c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-takgaimd/unsloth_82cb18b94a304bc9ba71989233d4ea2c
  Resolved https://github.com/unslothai/unsloth.git to commit f26d4e739ed507de7a9088da53d10fd02f58d160
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.7-py3-none-any.whl size=163138 sha256=e537eb3d377dc9875bdc59c940f460e9348a723a2e8a28f105dbe2a6a4d1a471
  Stored in directory: /tmp/pip-ephem-wheel-cache-v12s1vv7/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import json
from datasets import Dataset

# Function to load and process the new data format
def process_data(input_file):
    formatted_data = []
    with open(input_file, 'r') as f_in:
        data = json.load(f_in)  # Assuming the new file is a JSON array
        for example in data:
            instruction = example.get("instruction", "")
            input_text = example.get("input", "")
            output = example.get("output", "")

            # Use the instruction, input, and output fields directly
            alpaca_entry = {
                "text": f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
            }
            formatted_data.append(alpaca_entry)

    return formatted_data

# Specify your input JSON file
input_json = "/content/merged_dataset.json"

# Process the data
formatted_data = process_data(input_json)

# Create a Hugging Face Dataset from the formatted data
dataset = Dataset.from_dict({"text": [entry["text"] for entry in formatted_data]})

# Optionally, save the dataset to a local file for further use
dataset.save_to_disk("law_dataset_alpaca_format")

# Print dataset details
print(dataset)

# Now you can map it to your format
EOS_TOKEN = "<|endoftext|>"  # Define your EOS token
def formatting_prompts_func(examples):
    texts = examples["text"]
    formatted_texts = [text + EOS_TOKEN for text in texts]
    return {"text": formatted_texts}

# Apply the formatting function (if you want to prepare for training)
dataset = dataset.map(formatting_prompts_func, batched=True)

print("Dataset after formatting:")
print(dataset)


Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 2000
})


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset after formatting:
Dataset({
    features: ['text'],
    num_rows: 2000
})


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.581400
2,1.928800
3,1.999700
4,1.555000
5,1.428400
6,1.325000
7,1.103600
8,1.238500
9,1.036900
10,1.083400


In [ ]:
# alpaca_prompt = Copied from above
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Draft a hypothetical legal petition based on the provided case.", # instruction
        "Kesavananda Bharati v. State of Kerala (1973):This case arose when Kesavananda Bharati, a seer of a religious institution, challenged the Kerala state government’s attempts to control religious property, raising concerns over the state's power to amend the Constitution. The case became a major constitutional debate over whether Parliament could amend any part of the Constitution.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 500)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Draft a hypothetical legal petition based on the provided case.

### Input:
Kesavananda Bharati v. State of Kerala (1973):This case arose when Kesavananda Bharati, a seer of a religious institution, challenged the Kerala state government’s attempts to control religious property, raising concerns over the state's power to amend the Constitution. The case became a major constitutional debate over whether Parliament could amend any part of the Constitution.

### Response:
IN THE SUPREME COURT OF INDIA

Writ Petition (Civil) No. 164 of 1973

Kesavananda Bharati Sripadagiri, Petitioner

State Of Kerala, Respondent

PETITION UNDER ARTICLE 32 OF THE CONSTITUTION OF INDIA FOR DIRECTION TO THE STATE OF KERALA TO RESTORE AND MAINTAIN THE STATUS QUO AS PER THE PETITIONER'S SCHEDULED CASTE AND SCHEDULED TRIB

In [ ]:
pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 51.6 MB/s eta 0:00:00


In [ ]:
pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.1 MB/s eta 0:00:00


In [ ]:
# Install pdfplumber if not already installed
# !pip install pdfplumber

import pdfplumber
from google.colab import files

# Upload the PDF file
uploaded = files.upload()
extracted_text = ""
# Process the uploaded file
for file_name in uploaded.keys():
    print(f"Processing file: {file_name}")

    # Open and read the PDF file using pdfplumber
    with pdfplumber.open(file_name) as pdf:
        pdf_text = ""
        for page in pdf.pages:
            pdf_text += page.extract_text() + "\n"  # Adding newline between pages

        extracted_text+=pdf_text


# alpaca_prompt = Copied from above
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Draft a hypothetical legal petition based on the provided case.", # instruction
        extracted_text, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 500)

Saving ETHICS.pdf to ETHICS (3).pdf
Processing file: ETHICS (3).pdf


In [ ]:
model.save_pretrained("/content/drive/MyDrive/LlamaLawyer/LoraModel_Mergeddata") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/LlamaLawyer/LoraModel_Mergeddata")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/content/drive/MyDrive/LlamaLawyer/LoraModel_Mergeddata/tokenizer_config.json',
 '/content/drive/MyDrive/LlamaLawyer/LoraModel_Mergeddata/special_tokens_map.json',
 '/content/drive/MyDrive/LlamaLawyer/LoraModel_Mergeddata/tokenizer.json')

In [ ]:
pip install transformers peft torch


In [ ]:
!git clone https://github.com/unslothai/studio > /dev/null 2>&1
with open("studio/unsloth_studio/chat.py", "r") as chat_module: code = chat_module.read()

In [ ]:
exec(code)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import torch
from unsloth import FastLanguageModel

# Path to your saved model
model_path = "/content/drive/MyDrive/LlamaLawyer/LoraModel_Mergeddata"

# Load the fine-tuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")



ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
# Alpaca prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Format the prompt
prompt = alpaca_prompt.format(
    "Draft a hypothetical legal petition based on the provided case.",  # Instruction
    "Kesavananda Bharati v. State of Kerala (1973): This case arose when Kesavananda Bharati, a seer of a religious institution, challenged the Kerala state government’s attempts to control religious property, raising concerns over the state's power to amend the Constitution. The case became a major constitutional debate over whether Parliament could amend any part of the Constitution.",  # Input
    ""  # Output (leave blank for generation)
)

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Create a TextStreamer for streaming output
text_streamer = TextStreamer(tokenizer)

# Generate the output
outputs = model.generate(
    input_ids=inputs["input_ids"],  # Pass input_ids explicitly
    max_new_tokens=500,  # Limit output length
    temperature=0.7,  # Adjust creativity
    top_p=0.9,  # Nucleus sampling
    repetition_penalty=1.2,  # Penalize repeated tokens
    pad_token_id=tokenizer.pad_token_id,  # Ensure valid padding
    eos_token_id=tokenizer.eos_token_id,  # End-of-sequence token
    streamer=text_streamer,  # Stream the output
)

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/LlamaLawyer/LoraModel_Mergeddata", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is the punishment for drink and drive in India?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

NameError: name 'tokenizer' is not defined